In [25]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [26]:
q = pd.read_csv('/Users/shanewilliams/Quennect/src/q_data_cleaned2.csv')
f1sub = pd.read_csv('/Users/shanewilliams/Quennect/feb1substations.csv')

In [27]:
print(q.info)
print(q["poi_name"].nunique())

<bound method DataFrame.info of        Unnamed: 0      q_date  q_year  prop_year state entity utility  \
0               0    1/7/2022  2022.0     2024.0    MO    AEC     AEC   
1               3  11/19/2018  2018.0     2021.0    MO    AEC     AEC   
2               6  10/12/2017  2017.0     2020.0    MO    AEC     AEC   
3               8   5/31/2018  2018.0     2020.0    OK    AEC     AEC   
4               9   12/6/2018  2018.0     2021.0    MO    AEC     AEC   
...           ...         ...     ...        ...   ...    ...     ...   
11822       28086   5/31/2022  2022.0     2026.0    CO   TSGT    TSGT   
11823       28099   5/30/2022  2022.0     2025.0    CO   TSGT    TSGT   
11824       28100   5/31/2022  2022.0     2025.0    CO   TSGT    TSGT   
11825       28107   5/31/2022  2022.0     2025.0    CO   TSGT    TSGT   
11826       28124    5/9/2022  2022.0     2025.0    CO   TSGT    TSGT   

         county_1                    poi_name  region_CAISO  ...  type1_Steam  \
0      new

In [28]:
print(f1sub.info)
print(f1sub['name'].nunique())

<bound method DataFrame.info of         OID_  Join_Count  TARGET_FID  Join_Count_1  TARGET_FID_1  OBJECTID  \
0          1           1           9             1             9         9   
1          2           1          11             1            11        11   
2          3           1          12             1            12        12   
3          4           1          13             1            13        13   
4          5           1          14             1            14        14   
...      ...         ...         ...           ...           ...       ...   
58305  58306           1       58225             1         58225     58225   
58306  58307           1       58227             1         58227     58227   
58307  58308           1       58235             1         58235     58235   
58308  58309           1       58251             1         58251     58251   
58309  58310           1       58261             1         58261     58261   

       Join_Count_12  TARGET_FI

In [10]:
f1sub['tokens'] = f1sub['name'].str.lower().str.split()
q['tokens'] = q['poi_name'].str.lower().str.split()

def calc_similarity(tokens1, tokens2):
    set1 = set(tokens1)
    set2 = set(tokens2)
    common_tokens = set1.intersection(set2)
    total_tokens = set1.union(set2)
    return len(common_tokens) / len(total_tokens) if total_tokens else 0

best_match = defaultdict(lambda: {'name': None, 'score': 0})
for _, row1 in f1sub.iterrows():
    for _, row2 in q.iterrows():
        score = calc_similarity(row1['tokens'], row2['tokens'])
        if score > best_match[row1['name']]['score']:
            best_match[row1['name']] = {'name': row2['poi_name'], 'score': score}

matches_df = pd.DataFrame([{'name': k, 'matched_poi_name': v['name'], 'similarity_score': v['score']} for k, v in best_match.items()])

f1sub_matched = f1sub.merge(matches_df, on='name')
final_df = f1sub_matched.merge(q, left_on='matched_poi_name', right_on='poi_name', how='inner')

print(final_df.head())

    OID_  Join_Count  TARGET_FID  Join_Count_1  TARGET_FID_1  OBJECTID  \
0      1           1           9             1             9         9   
1   1567           1        5315             1          5315      5315   
2  15174           1           7             1             7         7   
3  18210           1        5311             1          5311      5311   
4  18211           1        5312             1          5312      5312   

   Join_Count_12  TARGET_FID_12  Join_Count_12_13  TARGET_FID_12_13  ...  \
0              1              9                 1                 9  ...   
1              1           5315                 1              5315  ...   
2              0              7                 1                 7  ...   
3              1           5311                 1              5311  ...   
4              1           5312                 1              5312  ...   

  type1_Waste Heat type1_Wind type1_Wood ia_status_Facility Study  \
0                0          1

In [29]:
print(f1sub['COUNTYNAME'].nunique())
print(q['county_1'].nunique())

1874
987


In [165]:
f1sub['tokens'] = f1sub['name'].str.replace('-', ' ').str.lower().str.split().str[0]
q['tokens'] = q['poi_name'].str.replace('-', ' ').str.lower().str.split().str[0]
q['state'] = q['state'].str.strip()
q['county_1'] = q['county_1'].str.strip().str.lower()
states = q['state'].unique()
f1sub['COUNTYNAME'] = f1sub['COUNTYNAME'].str.lower().str.strip()
q_small =  q.head(100)

In [133]:
#match by state and county
# q_joined = np.empty((0,87)) #initialize with col names maybe
# for state in states:
#     counties = q_small[q_small['state'] == str(state)]['county_1'].unique()
#     q_by_state = q_small[q_small['state'] == str(state)]
#     sub_by_state = f1sub[f1sub['STATE_ABBR'] == str(state)]
    
#     for county in counties:
#         q_by_county = q_by_state[q_by_state['county_1'] == county]
#         sub_by_county = sub_by_state[sub_by_state['COUNTYNAME'] == county]
        
#         for i in q_by_county.index:
#             for j in sub_by_county.index:
                
#                 if(q_by_county.loc[i, 'tokens'] == sub_by_county.loc[j, 'tokens']):
#                     row_to_append = np.array(sub_by_county.loc[j])
#                     dummy = np.concatenate((np.array(q_by_county.loc[i]), row_to_append))
#                     q_joined = np.append(q_joined, [dummy], axis=0)
        

[0 '1/7/2022' 2022.0 2024.0 'MO' 'AEC' 'AEC' 'new madrid'
 'New Madrid - Essex 345kV' 0 0 0 0 0 0 0 1 0 350.0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 'new' 20712 1 9357 1 9357 9357 0 9357
 1 9357 'way/168241735' 'New Madrid Substation'
 'Associated Electric Cooperative' 'Q6840876' 'substation' 'transmission'
 'multipolygon' 9357 36.51563499 -89.56768587 'MO' 'Missouri' 88.3
 'new madrid' 'New Madrid County' 'H1' 'County'
 'An active county or equivalent feature' nan nan nan nan nan nan 4.3 nan
 3.9469337 'new']
[6 '10/12/2017' 2017.0 2020.0 'MO' 'AEC' 'AEC' 'new madrid'
 'New Madrid Substation' 0 0 0 0 0 0 0 1 0 202.0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 'new' 20712 1 9357 1 9357 9357 0 9357 1
 9357 'way/168241735' 'New Madrid Substation'
 'Associated Electric Cooperative' 'Q6840876' 'substation' 'transmission'
 'multipolygon' 9357 36.51563499 -89.56768587 'MO' 'Missouri' 88.3
 'new madrid' 'New Madrid County' 'H1' 'County'
 'An active county or

In [166]:
q_joined = np.empty((0,87)) #initialize with col names maybe
for state in states:
    counties = q_small[q_small['state'] == str(state)]['county_1'].unique()
    q_by_state = q_small[q_small['state'] == str(state)]
    sub_by_state = f1sub[f1sub['STATE_ABBR'] == str(state)]
    
        
    for i in q_by_state.index:
        for j in sub_by_state.index:
            
            if(q_by_state.loc[i, 'tokens'] == sub_by_state.loc[j, 'tokens']):
                row_to_append = np.array(sub_by_state.loc[j])
                dummy = np.concatenate((np.array(q_by_state.loc[i]), row_to_append))
                q_joined = np.append(q_joined, [dummy], axis=0)
                break

In [159]:
qcols = q.columns
f1cols = f1sub.columns
col_names = np.append(qcols, f1cols)
q_total = pd.DataFrame(q_joined, columns = col_names)

In [160]:
q_total

,Unnamed: 0,q_date,q_year,prop_year,state,entity,utility,county_1,poi_name,region_CAISO,...,GEOID,votes_dem,votes_rep,votes_total,votes_per_sqkm,pct_dem_lead,solar_ann_ghi_rn,SymbolID,avg_wind_speed_meters_per_second,tokens
0,0,1/7/2022,2022.0,2024.0,MO,AEC,AEC,new madrid,New Madrid - Essex 345kV,0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.3,NaN,3.946934,new
1,3,11/19/2018,2018.0,2021.0,MO,AEC,AEC,pemiscot,New Madrid to Dell 500 kV,0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.3,NaN,3.946934,new
2,6,10/12/2017,2017.0,2020.0,MO,AEC,AEC,new madrid,New Madrid Substation,0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.3,NaN,3.946934,new
3,9,12/6/2018,2018.0,2021.0,MO,AEC,AEC,new madrid,New Madrid to Essex 345 kV,0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.3,NaN,3.946934,new
4,15,9/3/2020,2020.0,2023.0,MO,AEC,AEC,howard,Harrisburg - Higbee 161 kV,0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.3,NaN,3.251261,harrisburg
5,20,3/6/2019,2019.0,2019.0,MO,AEC,AEC,dunklin,St. Francis 161 kV,0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.2,NaN,3.59933,st.
6,24,8/22/2019,2019.0,2021.0,MO,AEC,AEC,new madrid,New Madrid 345 kV,0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.3,NaN,3.946934,new
7,31,11/19/2018,2018.0,2021.0,MO,AEC,AEC,pemiscot,New Madrid to Dell 500 kV,0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.3,NaN,3.946934,new
8,34,11/19/2018,2018.0,2021.0,MO,AEC,AEC,pemiscot,New Madrid to Dell 500 kV,0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.3,NaN,3.946934,new
9,38,12/6/2018,2018.0,2021.0,MO,AEC,AEC,new madrid,New Madrid to Essex 345 kV,0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.3,NaN,3.946934,new


In [167]:
q_joined.shape

(44, 87)

In [158]:
q_small.shape

(100, 49)